<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#스파크-데이터-타입으로-변환" data-toc-modified-id="스파크-데이터-타입으로-변환-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>스파크 데이터 타입으로 변환</a></span></li><li><span><a href="#Boolean-데이터-타입-다루기" data-toc-modified-id="Boolean-데이터-타입-다루기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Boolean 데이터 타입 다루기</a></span></li><li><span><a href="#수치형-데이터-타입-다루기" data-toc-modified-id="수치형-데이터-타입-다루기-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>수치형 데이터 타입 다루기</a></span></li><li><span><a href="#문자열-데이터-타입-다루기" data-toc-modified-id="문자열-데이터-타입-다루기-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>문자열 데이터 타입 다루기</a></span><ul class="toc-item"><li><span><a href="#문자열---정규표현식" data-toc-modified-id="문자열---정규표현식-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>문자열 - 정규표현식</a></span></li></ul></li></ul></div>

In [1]:
!ls /Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/retail-data/

all    by-day


In [2]:
df = spark.read.format('csv')\
     .option('header', 'true')\
     .option('inferSchema', 'true')\
     .load('/Users/younghun/Desktop/gitrepo/data/spark_perfect_guide/retail-data/by-day/2010-12-01.csv')
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [3]:
# 트랜스포메이션 결과를 확인하기 위해 임시테이블인 뷰로 등록
df.createOrReplaceTempView('dfTable')

In [4]:
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



## 스파크 데이터 타입으로 변환
- 스파크 자체 데이터 타입으로 변환시키기 위해 ``lit`` 함수를 분명히 기억!!
    * ``lit``은 다른 언어의 데이터 타입을 스파크 데이터 타입에 맞게 변환함

In [5]:
from pyspark.sql.functions import lit

test = df.select(lit(5), lit('five'), lit(5.0))
test.show(5)

+---+----+---+
|  5|five|5.0|
+---+----+---+
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
+---+----+---+
only showing top 5 rows



## Boolean 데이터 타입 다루기
- Boolean은 로우를 필터링할 때 매우 자주 사용됨
- 주로 조건문이 일치하면 True, 일치하지 않으면 False로 반환시켜 필터링 함

In [6]:
df.show(2)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [7]:
from pyspark.sql.functions import col

df.where(col('InvoiceNo') != 536365)\
  .select("InvoiceNo", "Description")\
  .show(5, truncate=False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [8]:
df.where(col('InvoiceNo') != 536365)\
  .selectExpr("InvoiceNo","Description")\
  .show(5, truncate=False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import expr
# 문자열 표현식으로도 사용 가능
df.where("InvoiceNo != 536365")\
  .select(expr("InvoiceNo"), expr("Description"))\
  .show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



- ``instr(string, substr)``: ``string`` 문자열에서 ``substr``이 시작하는 첫 번째 인덱스를 반환(단, **Pyspark에서 문자열 index 번호는 1부터 시작!**)

In [12]:
# 여러가지 필터링 조건
from pyspark.sql.functions import instr

priceFilter = col('UnitPrice') > 600
descriFilter = instr(df.Description, 'POSTAGE') >= 1

# 논리연산자 사용가능
"""
and로 필터추가하려면 연속적으로 where 구문 붙여서 하나의 문장으로 만들면 됨
or(|)로 필터추가하려면 동일한 구문 조건 안에 넣어야 함(하단 참조)
"""
df.where(df.StockCode.isin("DOT")).where(priceFilter | descriFilter).show(5)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



- ``select``시, 불리언 타입의 칼럼을 추출할 때는 자동으로 True인 값들만 추출함(Boolean Indexing 처럼!)

In [13]:
# 불리언 표현식을 사용해서 DF 필터링하기
DOTCodeFilter = col('StockCode') == 'DOT'
priceFilter = col('UnitPrice') >= 600
descriFilter = instr(df.Description, 'POSTAGE') >= 1

df.withColumn('isExpensive', DOTCodeFilter & (priceFilter | descriFilter))\
  .where('isExpensive')\
  .select('UnitPrice', 'isExpensive').show()

+---------+-----------+
|UnitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



- 위와 같이 필터조건을 표현식으로 작성해도 되지만 **칼럼명을 사용**해서도 작성할 수 있음
    * 동작방식은 동일하기 때문에 표현식과 칼럼명 방법 사이의 속도 차이 존재 X

In [14]:
from pyspark.sql.functions import expr

df.withColumn('isExpensive', expr('NOT UnitPrice <= 250'))\
  .where('isExpensive')\
  .select('Description', 'isExpensive').show()

+--------------+-----------+
|   Description|isExpensive|
+--------------+-----------+
|DOTCOM POSTAGE|       true|
|DOTCOM POSTAGE|       true|
+--------------+-----------+



- 불리언 표현식을 만들 때 ``null`` 데이터는 어떻게 다루어야 할까?
    * 다음과 같은 코드 수행
- ``join``방법 중 ``left-anti join``이란 왼쪽 테이블에는 있으면서 오른쪽 테이블에는 없는 row들만 남김

In [15]:
df.where(col('Description').eqNullSafe('hello')).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



- ``eqNullSafe(df.column)`` 메소드는 결측치를 없는 데이터로 간주함!

## 수치형 데이터 타입 다루기
- ``pow``를 활용해 거듭제곱 계산 가능(=Native Python)

In [20]:
from pyspark.sql.functions import expr, pow

fabricatedQuantity = pow(col('Quantity')*col('UnitPrice'), 2) + 5 # Column으로 반환됨
df.select(expr('CustomerId'), fabricatedQuantity.alias('realQuantity')).show(5)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 5 rows



In [21]:
# SQL 구문으로도 가능
df.selectExpr("CustomerId",
             "(POWER((Quantity * UnitPrice), 2) + 5) as realQuantity").show(5)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 5 rows



- 반올림 메소드: ``round()``
- 내림 메소드: ``bround()``

In [24]:
from pyspark.sql.functions import lit, round, bround

# lit으로 스파크 자체 데이터 타입으로 변경
df.select(round(lit(2.5)), bround(lit(2.5))).show(5)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 5 rows



- 두 수치형 변수간의 ``Pearson Correlation`` 계산하기

In [26]:
from pyspark.sql.functions import corr

df.stat.corr('Quantity', 'UnitPrice')

-0.04112314436835551

In [27]:
df.select(corr('Quantity', 'UnitPrice')).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



- 하나 이상의 수치형 칼럼에 대한 요약 통계 계산 방법
    * ``describe()`` 메서드 사용. 단, 통계 스키마가 변경될 수도 있으니 콘솔용에서 확인 가능
    * 정확한 통계수치를 보기 위해서는 통계 요약값 계산 메소드를 임포트

In [36]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

In [37]:
from pyspark.sql.functions import count, max, min, mean, stddev_pop

- StatFunctions 패키지 활용해서도 가능. ``stat``속성으로 접근해서 사용가능한 DataFrame 메서드
    * ex) ``approxQuantile(column, probability, relativeError)``
        - ``probability``: 백분위(list/tuple)
        - ``relativeError``: 목표값과 얼마만큼의 에러차이를 허용할 것인지(만약 0으로 설정시 목표치와 매우 근사하지만 연산 비용이 많이듬(expensive)

In [41]:
df.stat.approxQuantile('UnitPrice', [0.5], 0.05)

[2.51]

- StatFunctions는 ``crosstab``(교차표) 기능도 제공

In [ ]:
df.stat.crosstab('StockCode', 'Quantity').show()

In [45]:
df.stat.freqItems(['StockCode', 'Quantity']).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



- StatFunctions의 ``monotonically_increasing_id()``: 모든 로우에 unique한 ID값을 만들어줌

In [46]:
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show(5)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
+-----------------------------+
only showing top 5 rows



- 이외 StatFunctions에는 다양한 고급 알고리즘 메소드도 제공을 함. 필요 시 공식문서 참조하기

## 문자열 데이터 타입 다루기

- ``initcap``: 주어진 문자열에서 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변경
- ``upper``: 모든 문자를 대문자로
- ``lower``: 모든 문자를 소문자로

In [49]:
from pyspark.sql.functions import initcap

df.select(initcap(col('Description'))).show(5, truncate=False)

+-----------------------------------+
|initcap(Description)               |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
|Red Woolly Hottie White Heart.     |
+-----------------------------------+
only showing top 5 rows



In [51]:
from pyspark.sql.functions import upper, lower

df.select(col('Description'),
         lower(col('Description')),
         upper(col('Description'))).show(5, truncate=False)

+-----------------------------------+-----------------------------------+-----------------------------------+
|Description                        |lower(Description)                 |upper(Description)                 |
+-----------------------------------+-----------------------------------+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |white hanging heart t-light holder |WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |white metal lantern                |WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |cream cupid hearts coat hanger     |CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|knitted union flag hot water bottle|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |red woolly hottie white heart.     |RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+-----------------------------------+-----------------------------------+
only showi

- 공백 제거하거나 추가하는 작업도 가능
    * ``ltrim, rtrim, lpad, rpad, trim``

In [60]:
from pyspark.sql.functions import lit, ltrim, rtrim, lpad, rpad, trim

hello_col = lit('  HELLO  ')
df.select(ltrim(hello_col),
         rtrim(hello_col),
         lpad(lit('HELLO'), 15, '#'),
         rpad(lit('HELLO'), 15, '$')).show()

+----------------+----------------+------------------+------------------+
|ltrim(  HELLO  )|rtrim(  HELLO  )|lpad(HELLO, 15, #)|rpad(HELLO, 15, $)|
+----------------+----------------+------------------+------------------+
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   ##########HELLO|   HELLO$$$$$$$$$$|
|         HELLO  |           HELLO|   

### 문자열 - 정규표현식